In [14]:
import pandas as pd
import datetime
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#pd.set_option('display.max_info_columns', 150)
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)


In [15]:
pd.set_option('display.max_info_columns', 150)
pd.set_option('display.max_columns', None)

In [16]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))/10**6) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('total', 261.835295),
 ('NamespaceMagics', 0.000888),
 ('var_dic_list', 0.000136),
 ('np', 8e-05),
 ('pd', 8e-05),
 ('plt', 8e-05),
 ('sns', 8e-05),
 ('getsizeof', 7.2e-05)]

# 데이터 불러오기

In [17]:
total = pd.read_csv('./data_merged/final_dataset2.csv', dtype={'customer_zip_code_prefix':str,'seller_zip_code_prefix':str},
                    parse_dates = ['order_purchase_timestamp', 'order_aproved_at', 'order_estimated_delivery_date',
                                   'order_delivered_customer_date', 'review_creation_date', 'review_answer_timestamp'],
                   na_values = '')

# 데이터 조회

In [18]:
total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93000 entries, 0 to 92999
Data columns (total 122 columns):
order_id                         93000 non-null object
order_status                     93000 non-null object
order_products_value             93000 non-null float64
order_freight_value              93000 non-null float64
order_items_qty                  93000 non-null int64
order_sellers_qty                93000 non-null int64
order_purchase_timestamp         93000 non-null datetime64[ns]
order_aproved_at                 92982 non-null datetime64[ns]
order_estimated_delivery_date    93000 non-null datetime64[ns]
order_delivered_customer_date    90705 non-null datetime64[ns]
product_id                       93000 non-null object
product_category_name_english    93000 non-null object
product_name_lenght              93000 non-null int64
product_description_lenght       93000 non-null int64
product_photos_qty               93000 non-null int64
product_weight_g                 929

# 데이터 전처리

## - 도시 이름과 주 소문자로 통일 

In [19]:
total['customer_state'] = total['customer_state'].apply(lambda x : x.lower())
total['customer_city'] = total['customer_city'].apply(lambda x : x.lower())
total['seller_state'] = total['seller_state'].apply(lambda x : x.lower() if pd.notnull(x) else x)
total['seller_city'] = total['seller_city'].apply(lambda x : x.lower() if pd.notnull(x) else x)

# 파생변수 생성

## 파생변수 1. 'pn_review_score' : review score positive / negative

In [20]:
total['pn_review_score']= ['Positive' if i>=4 else 'Negative' for i in total['review_score']]

## 파생변수 2. 'del_period' : 절대적 배송 기간

In [21]:
total['del_period'] = (total['order_delivered_customer_date']-total['order_purchase_timestamp'])/datetime.timedelta(days=1)

In [22]:
# 이영 ver
'''
abs_deli_days = total['order_delivered_customer_date']-total['order_purchase_timestamp']
scaled_days1 = list(map(lambda abs_deli_days: abs_deli_days.total_seconds()/datetime.timedelta(days=1).total_seconds(), abs_deli_days))
total['del_period'] = scaled_days1

'''

"\nabs_deli_days = total['order_delivered_customer_date']-total['order_purchase_timestamp']\nscaled_days1 = list(map(lambda abs_deli_days: abs_deli_days.total_seconds()/datetime.timedelta(days=1).total_seconds(), abs_deli_days))\ntotal['del_period'] = scaled_days1\n\n"

In [23]:
total['del_period'].head()

0     5.739851
1    12.588803
2     7.957293
3     5.293556
4    11.067474
Name: del_period, dtype: float64

## 파생변수 3. 'del_period_psy' : 심리적 배송 기간

In [24]:
total['del_period_psy'] = (total['order_estimated_delivery_date']-total['order_delivered_customer_date'])/datetime.timedelta(days=1)
total['del_period_psy'].head()

0    36.538609
1    10.882713
2     6.061133
3     8.326536
4     5.278568
Name: del_period_psy, dtype: float64

In [25]:
# 이영이 버전
'''
psy_deli_days = total['order_estimated_delivery_date']-total['order_delivered_customer_date']
total['del_period_psy'] = psy_deli_days 
scaled_days2 = list(map(lambda abs_deli_days: abs_deli_days.total_seconds()/datetime.timedelta(days=1).total_seconds(), psy_deli_days))
scaled_days2[:6]
total['del_period_psy'] = scaled_days2 
'''

"\npsy_deli_days = total['order_estimated_delivery_date']-total['order_delivered_customer_date']\ntotal['del_period_psy'] = psy_deli_days \nscaled_days2 = list(map(lambda abs_deli_days: abs_deli_days.total_seconds()/datetime.timedelta(days=1).total_seconds(), psy_deli_days))\nscaled_days2[:6]\ntotal['del_period_psy'] = scaled_days2 \n"

### abs & psy 파생변수 시각화

## 파생변수 4. 'del_period_deadline_yn' : 배송기한준수여부

In [26]:
total['del_period_deadline_yn'] = [True if i>=0 else False for i in total['del_period_psy']]

In [27]:
# 이영 ver
'''
psy_scaled_days = list(map(lambda psy_deli_days: psy_deli_days.total_seconds()/datetime.timedelta(days=1).total_seconds(), psy_deli_days))
del_period_psy = pd.concat([pd.DataFrame(psy_scaled_days),total['review_score']],axis=1)
del_period_psy.columns = ['del_period_psy','review_score']
del_period_psy.head()

del_period_deadline =[True if i>=0 else False for i in del_period_psy['del_period_psy']]
total['del_period_deadline_yn']=del_period_deadline 
'''

"\npsy_scaled_days = list(map(lambda psy_deli_days: psy_deli_days.total_seconds()/datetime.timedelta(days=1).total_seconds(), psy_deli_days))\ndel_period_psy = pd.concat([pd.DataFrame(psy_scaled_days),total['review_score']],axis=1)\ndel_period_psy.columns = ['del_period_psy','review_score']\ndel_period_psy.head()\n\ndel_period_deadline =[True if i>=0 else False for i in del_period_psy['del_period_psy']]\ntotal['del_period_deadline_yn']=del_period_deadline \n"

## 파생변수 5. 'payment_voucher_yn' : 쿠폰사용여부

In [28]:
total.columns.get_loc('payment_type_1')

58

In [29]:
total.iloc[:,58:] == 'voucher'

,payment_type_1,payment_type_2,payment_type_3,payment_type_4,payment_type_5,payment_type_6,payment_type_7,payment_type_8,payment_type_9,payment_type_10,payment_type_11,payment_type_12,payment_type_13,payment_type_14,payment_type_15,payment_type_16,payment_type_17,payment_type_18,payment_type_19,payment_type_20,payment_type_21,payment_type_22,payment_type_23,payment_type_24,payment_type_25,payment_type_26,payment_type_27,payment_type_28,payment_type_29,value_1,value_2,value_3,value_4,value_5,value_6,value_7,value_8,value_9,value_10,value_11,value_12,value_13,value_14,value_15,value_16,value_17,value_18,value_19,value_20,value_21,value_22,value_23,value_24,value_25,value_26,value_27,value_28,value_29,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,pn_review_score,del_period,del_period_psy,del_period_deadline_yn
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fals

In [30]:
total['payment_voucher_yn'] = (total.iloc[:,58:] == 'voucher').any(axis=1)

## 파생변수 6. 'freight_value_proportion' : 배송료 비중

In [31]:
total['freight_value_proportion'] = total['order_freight_value']/total['order_products_value']
total['freight_value_proportion'].head()

0    0.039554
1    0.348500
2    0.136767
3    0.198847
4    0.033490
Name: freight_value_proportion, dtype: float64

In [32]:
# 이영 ver
'''
cost = pd.concat([total['order_products_value'],total['order_freight_value'],total['review_score']],axis=1)
cost.columns=['price','freight','review_score']
cost['freight_price']=cost['freight']/cost['price']
total['freight_value_proportion'] = cost['freight_price']

'''

"\ncost = pd.concat([total['order_products_value'],total['order_freight_value'],total['review_score']],axis=1)\ncost.columns=['price','freight','review_score']\ncost['freight_price']=cost['freight']/cost['price']\ntotal['freight_value_proportion'] = cost['freight_price']\n\n"

## 파생변수 7.'order_time_6' : 주문 시간대 6시간 단위

In [33]:
opt = total.order_purchase_timestamp.dt

opt_prop = pd.DataFrame({'year':opt.year,
             'month':opt.month,
             'day':opt.day,
             'hour':opt.hour,
             'minute':opt.minute,
              #'week':opt.week,
              'weekofyear':opt.weekofyear,
             'dayofyear':opt.dayofyear,
              'weekday':opt.weekday,
              'quarter':opt.quarter
             })
hour_6 = ['time_%d'%(i//6+1) for i in opt_prop['hour']]
hour_6 = pd.concat([pd.DataFrame(hour_6,columns=['time']),total['review_score']],axis=1)
total['order_time_6'] = hour_6['time']

## 파생변수 8. 'order_day' : 주문 요일

In [34]:
weekday_mapping = {
    0 : 'Monday',
    1 : 'Tuesday',
    2 : 'Wednesday',
    3 : 'Thursday',
    4 : 'Friday',
    5 : 'Saturday',
    6 : 'Sunday'
}

opt_prop['weekday_str'] = opt_prop['weekday'].map(lambda x : weekday_mapping.get(x))

In [35]:
total['order_day'] = opt_prop['weekday_str']


## 파생변수 9. 'order_week_day_end' : 주문 평일 주말

In [36]:
order_week_day_end = [0 if i=='Saturday'or i=='Sunday' else 1 for i in opt_prop['weekday_str']]
total['order_week_day_end']=order_week_day_end

## 파생변수 10. 'review_ans_time_6' : 리뷰작성시간대

In [37]:
apt = total.review_answer_timestamp.dt

apt_prop = pd.DataFrame({'year':apt.year,
             'month':apt.month,
             'day':apt.day,
             'hour':apt.hour,
             'minute':apt.minute,
              #'week':opt.week,
              'weekofyear':apt.weekofyear,
             'dayofyear':apt.dayofyear,
              'weekday':apt.weekday,
              'quarter':apt.quarter
             })
ans_hour_6 = ['time_%d'%(i//6+1) for i in apt_prop['hour']]
ans_hour_6 = pd.concat([pd.DataFrame(ans_hour_6,columns=['time']),total['review_score']],axis=1)
total['review_ans_time_6'] = ans_hour_6['time']

## 파생변수 11. 'review_ans_day' : 리뷰작성요일

In [38]:
weekday_mapping = {
    0 : 'Monday',
    1 : 'Tuesday',
    2 : 'Wednesday',
    3 : 'Thursday',
    4 : 'Friday',
    5 : 'Saturday',
    6 : 'Sunday'
}

apt_prop['weekday_str'] = apt_prop['weekday'].map(lambda x : weekday_mapping.get(x))
total['review_ans_day'] = apt_prop['weekday_str']

## 파생변수 12. 'review_ans_w_d_e' : 리뷰작성평일주말

In [39]:
review_ans_w_d_e = [0 if i=='Saturday'or i=='Sunday' else 1 for i in apt_prop['weekday_str']]
total['review_ans_w_d_e'] =review_ans_w_d_e

## 파생변수 14. 'installments_yn' : 할부여부

In [40]:
total.columns.tolist()

['order_id',
 'order_status',
 'order_products_value',
 'order_freight_value',
 'order_items_qty',
 'order_sellers_qty',
 'order_purchase_timestamp',
 'order_aproved_at',
 'order_estimated_delivery_date',
 'order_delivered_customer_date',
 'product_id',
 'product_category_name_english',
 'product_name_lenght',
 'product_description_lenght',
 'product_photos_qty',
 'product_weight_g',
 'product_length_cm',
 'product_height_cm',
 'product_width_cm',
 'customer_unique_id',
 'customer_id',
 'customer_state',
 'customer_city',
 'customer_zip_code_prefix',
 'seller_id',
 'seller_state',
 'seller_city',
 'seller_zip_code_prefix',
 'sequential',
 'installments_1',
 'installments_2',
 'installments_3',
 'installments_4',
 'installments_5',
 'installments_6',
 'installments_7',
 'installments_8',
 'installments_9',
 'installments_10',
 'installments_11',
 'installments_12',
 'installments_13',
 'installments_14',
 'installments_15',
 'installments_16',
 'installments_17',
 'installments_18',
 'i

In [41]:
total.columns.get_loc('installments_1')

29

In [42]:
total.columns.get_loc('installments_29')

57

In [43]:
total.iloc[:,29:58]

,installments_1,installments_2,installments_3,installments_4,installments_5,installments_6,installments_7,installments_8,installments_9,installments_10,installments_11,installments_12,installments_13,installments_14,installments_15,installments_16,installments_17,installments_18,installments_19,installments_20,installments_21,installments_22,installments_23,installments_24,installments_25,installments_26,installments_27,installments_28,installments_29
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
total['installments_yn'] = (total.iloc[:,29:58]>1).any(axis=1)

In [45]:
#installments_yn = [1 if i>1 else 0 for i in installments_num]
#total['installments_yn'] =installments_yn

## 파생변수 13. 'sim_installments_mean' : 단순 평균 할부개월

In [46]:
installments_total_num = [0 for i in range(len(total))]
for num in range(1,4):
    installments_cnt =[ i if pd.notnull(i) else 0 for i in total['installments_%d'%num]] # installments에 NaN/Null : Voucher라는 뜻
    installments_total_num = [x + y for x,y in zip(installments_cnt,installments_total_num)]
installments_num = [x/y for x,y in zip(installments_total_num,total['sequential'])]
total['sim_installments_mean']=installments_num

In [47]:
total['sim_installments_mean'].loc[total['sim_installments_mean'] < 0.9, ]

998      0.600000
1151     0.272727
1320     0.428571
3761     0.750000
3915     0.750000
4148     0.750000
4849     0.375000
4908     0.500000
4910     0.750000
4945     0.500000
6416     0.750000
6631     0.600000
6926     0.600000
7484     0.600000
7487     0.600000
7574     0.600000
7837     0.750000
8659     0.600000
9192     0.750000
11059    0.750000
12353    0.500000
12553    0.750000
12595    0.750000
13833    0.750000
13906    0.750000
15454    0.375000
15696    0.750000
15848    0.750000
16247    0.750000
16561    0.157895
           ...   
85590    0.600000
85888    0.157895
86236    0.750000
86401    0.750000
86516    0.750000
86739    0.750000
87539    0.750000
88006    0.750000
88334    0.375000
89155    0.428571
89262    0.272727
89352    0.500000
89388    0.214286
89633    0.583333
89877    0.500000
89923    0.750000
90038    0.750000
90306    0.200000
90625    0.428571
90864    0.428571
90998    0.500000
91223    0.500000
91409    0.230769
91420    0.750000
91537    0

## 파생변수 15. 'wgt_installments_mean' : 가중 평균 할부 개월

### payment type가 voucher 아닌 것들의 sum( installment * value )

In [48]:
weighted_install = [0 for i in range(len(total))]
for num in range(1,30):
    install_vsum =[total['value_%d'%num].iloc[ind] * total['installments_%d'%num].iloc[ind] if (i!='voucher' and pd.notnull(i)) else 0 for ind,i in enumerate(total['payment_type_%d'%num])]
    weighted_install = [ x+y for x,y in zip(install_vsum,weighted_install)]

### payment type가 voucher 아닌 것들의 value sum

In [49]:

sum_nvoucher_value = [0 for i in range(len(total))]
for num in range(1,30):
    temp =[total['value_%d'%num].iloc[ind] if (i!='voucher' and pd.notnull(i)) else 0 for ind,i in enumerate(total['payment_type_%d'%num])]
    sum_nvoucher_value = [x + y for x,y in zip(temp,sum_nvoucher_value)]


### weighted_install / sum_nvoucher_value

In [50]:
wgt_installments_mean = [x/y if y!=0 else np.nan for x,y in zip(weighted_install,sum_nvoucher_value)]
total['wgt_installments_mean']=wgt_installments_mean


In [51]:
total[['sim_installments_mean','wgt_installments_mean']].head(30)

,sim_installments_mean,wgt_installments_mean
0,1.0,1.0
1,1.0,1.0
2,1.0,1.0
3,5.0,5.0
4,10.0,10.0
5,2.0,2.0
6,1.5,2.0
7,1.0,1.0
8,2.0,2.0
9,3.0,3.0


In [52]:
total.loc[total['wgt_installments_mean']< 0.9, ]

,order_id,order_status,order_products_value,order_freight_value,order_items_qty,order_sellers_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,product_id,product_category_name_english,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_unique_id,customer_id,customer_state,customer_city,customer_zip_code_prefix,seller_id,seller_state,seller_city,seller_zip_code_prefix,sequential,installments_1,installments_2,installments_3,installments_4,installments_5,installments_6,installments_7,installments_8,installments_9,installments_10,installments_11,installments_12,installments_13,installments_14,installments_15,installments_16,installments_17,installments_18,installments_19,installments_20,installments_21,installments_22,installments_23,installments_24,installments_25,installments_26,installments_27,installments_28,installments_29,payment_type_1,payment_type_2,payment_type_3,payment_type_4,payment_type_5,payment_type_6,payment_type_7,payment_type_8,payment_type_9,payment_type_10,payment_type_11,payment_type_12,payment_type_13,payment_type_14,payment_type_15,payment_type_16,payment_type_17,payment_type_18,payment_type_19,payment_type_20,payment_type_21,payment_type_22,payment_type_23,payment_type_24,payment_type_25,payment_type_26,payment_type_27,payment_type_28,payment_type_29,value_1,value_2,value_3,value_4,value_5,value_6,value_7,value_8,value_9,value_10,value_11,value_12,value_13,value_14,value_15,value_16,value_17,value_18,value_19,value_20,value_21,value_22,value_23,value_24,value_25,value_26,value_27,value_28,value_29,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,pn_review_score,del_period,del_period_psy,del_period_deadline_yn,payment_voucher_yn,freight_value_proportion,order_time_6,order_day,order_week_day_end,review_ans_time_6,review_ans_day,review_ans_w_d_e,installments_yn,sim_installments_mean,wgt_installments_mean
43510,1a57108394169c0b47d8f876acc9ba2d,delivered,83.38,46.56,2,1,2018-05-15 16:25:14,2018-05-15 16:36:52.298005,2018-06-06,2018-05-24 15:45:41.989939,db35a562fb6ba63e19fa42a15349dc04,housewares,56,235,1,500.0,23.0,11.0,23.0,cb0d53af4a4a766f0573c1453a2125e1,48ebb06cf56dba9d009230cc751bb195,ba,feira de santana,440,282f23a9769b2690c5dda22e316f9941,mg,belo horizonte,315,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,credit_card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,129.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1a57108394169c0b47d8f876acc9ba2d,5,NaN,NaN,2018-05-25,2018-05-26 03:24:28,Positive,8.972546,12.343264,True,False,0.558407,time_3,Tuesday,1,time_1,Saturday,0,False,0.0,0.0
49640,744bade1fcf9ff3f31d860ace076d422,delivered,45.90,12.79,1,1,2018-04-22 11:34:42,2018-04-24 19:04:46.235669,2018-05-16,2018-04-27 20:55:28.630797,0cf573090c66bb30ac5e53c82bdb0403,telephony,26,119,1,150.0,16.0,4.5,13.0,00b56e8ececa84d3a1fd1489818da6a8,5e5794daaa13f73e2f1cdb4114529843,sp,sao paulo,052,7202e2ba20579a9bd1acb29e61fe71f6,sp,franca,144,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,credit_card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,744bade1fcf9ff3f31d860ace076d422,5,NaN,NaN,2018-04-28,2018-04-29 20:43:50,Positive,5.389429,18.128141,True,False,0.278649,time_2,Sunday,0,time_4,Sunday,0,False,0.0,0.0


## 파생변수 16. 'review_ans_period' : 리뷰응답신속도 

In [ ]:
total['review_ans_period']= total['review_answer_timestamp'] - total['review_creation_date']

In [ ]:
review_ans_period = total['review_ans_period']

In [ ]:
scaled_days = list(map(lambda review_ans_period: review_ans_period.total_seconds()/datetime.timedelta(days=1).total_seconds(), review_ans_period))

In [ ]:
total['review_ans_period'] = scaled_days

In [ ]:
total[['order_id','review_ans_period','review_score']].head()

,order_id,review_ans_period,review_score
0,b95df3cef5297e79ef709ba256518f6f,2.109456,5
1,8a723730400b508cbf47fbef4a76ec8e,3.120718,5
2,cf71b9b6b7ed12d72f541b0420023f19,2.824965,4
3,93ef9253c47e7dfc633c01f728488cb1,2.859444,4
4,35bc592a16d6706d14d29f07ed1d13c1,1.696319,5


## 파생변수 17. 'review_comment_yn' : 리뷰 작성 여부

In [ ]:
#is_title = [True if pd.notnull(i) else False for i in total['review_comment_title']]
review_comment_yn = [True if pd.notnull(i) else False for i in total['review_comment_message']]

In [ ]:
total['review_comment_yn']=review_comment_yn

## 파생변수 18. 'seller_response_time' : 판매자 응답 속도

In [ ]:
td_seller_response = total['order_aproved_at'] - total['order_purchase_timestamp']

In [ ]:
scaled_days = list(map(lambda td_seller_response: td_seller_response.total_seconds()/datetime.timedelta(days=1).total_seconds(), td_seller_response))

In [ ]:
total['seller_response_time']=scaled_days

In [ ]:
total['seller_response_time'].head()

0    0.390516
1    0.045551
2    1.141955
3    1.005649
4    0.009416
Name: seller_response_time, dtype: float64

## 파생변수 19. 지도정보

> 모두 geolocation 파일의 prefix 기준으로 customer, seller 각각의 lat, lng 추가함 

In [ ]:
dir = "./data_version2/"
orders = pd.read_csv(dir + 'olist_public_dataset_v2.csv')
geoloc = pd.read_csv('./data_version1/olist_geolocation_dataset.csv',dtype={'geolocation_zip_code_prefix':str})
sellers = pd.read_csv(dir + 'sellers_olist_public_dataset_.csv')
customers = pd.read_csv(dir + 'olist_public_dataset_v2_customers.csv')

'customer_city', 'customer_state', 'customer_zip_code_prefix','seller_zip_code_prefix','seller_city', 'seller_state'

### - geoloc 도시와 주 소문자로  변경

In [ ]:
geoloc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
geolocation_zip_code_prefix    1000163 non-null object
geolocation_lat                1000163 non-null float64
geolocation_lng                1000163 non-null float64
geolocation_city               1000163 non-null object
geolocation_state              1000163 non-null object
dtypes: float64(2), object(3)
memory usage: 38.2+ MB


In [ ]:
geoloc.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,01037,-23.545621,-46.639292,sao paulo,SP
1,01046,-23.546081,-46.644820,sao paulo,SP
2,01046,-23.546129,-46.642951,sao paulo,SP
3,01041,-23.544392,-46.639499,sao paulo,SP
4,01035,-23.541578,-46.641607,sao paulo,SP


In [ ]:
geoloc.columns=['zip_code_prefix','lat','lng','city','state']

In [ ]:
geoloc['state'] = geoloc['state'].apply(lambda x : x.lower())
geoloc['city'] = geoloc['city'].apply(lambda x : x.lower())

In [ ]:
geoloc['zip_code_prefix']=[i[:3] for i in geoloc['zip_code_prefix']] #5글자짜리 3글자로 변경

In [ ]:
geoloc.head() # 확인

,zip_code_prefix,lat,lng,city,state
0,010,-23.545621,-46.639292,sao paulo,sp
1,010,-23.546081,-46.644820,sao paulo,sp
2,010,-23.546129,-46.642951,sao paulo,sp
3,010,-23.544392,-46.639499,sao paulo,sp
4,010,-23.541578,-46.641607,sao paulo,sp


### - geoloc customer 의 지도 정보와 병합
> geoloc의 zip code prefix를 기준으로 평균값과 customer zip code prefix와 대응시킴 

In [ ]:
geoloc_customer = geoloc.groupby('zip_code_prefix').mean()
geoloc_customer.head()

,lat,lng
zip_code_prefix,,
010,-23.545414,-46.637157
011,-23.526827,-46.650764
012,-23.541328,-46.656093
013,-23.557785,-46.648632
014,-23.568757,-46.666038


In [ ]:
geoloc_customer.reset_index(inplace=True)
geoloc_customer.rename(columns={'zip_code_prefix':'customer_zip_code_prefix','lat': 'cus_lat','lng':'cus_lng'}, inplace=True)
geoloc_customer.head()

,customer_zip_code_prefix,cus_lat,cus_lng
0,010,-23.545414,-46.637157
1,011,-23.526827,-46.650764
2,012,-23.541328,-46.656093
3,013,-23.557785,-46.648632
4,014,-23.568757,-46.666038


In [ ]:
total = total.merge(geoloc_customer,on='customer_zip_code_prefix', how='left')

### 결측치 확인 결과 2개가 매칭되지 않음

In [ ]:
total[pd.isnull(total['cus_lat'])][['customer_zip_code_prefix','customer_city','customer_state','cus_lat','cus_lng']]

,customer_zip_code_prefix,customer_city,customer_state,cus_lat,cus_lng
34246,410,salvador,ba,NaN,NaN
75616,201,rio de janeiro,rj,NaN,NaN


### prefix 410 --> 411 
### prefix 201 --> 202 로 결측치 대체

In [ ]:
total.loc[total.customer_zip_code_prefix=='410','cus_lat'] = geoloc_customer[geoloc_customer['customer_zip_code_prefix']=='411']['cus_lat'].iloc[0]
total.loc[total.customer_zip_code_prefix=='410','cus_lng'] = geoloc_customer[geoloc_customer['customer_zip_code_prefix']=='411']['cus_lng'].iloc[0]

In [ ]:
total.loc[total.customer_zip_code_prefix=='201','cus_lat'] = geoloc_customer[geoloc_customer['customer_zip_code_prefix']=='202']['cus_lat'].iloc[0]
total.loc[total.customer_zip_code_prefix=='201','cus_lng'] = geoloc_customer[geoloc_customer['customer_zip_code_prefix']=='202']['cus_lng'].iloc[0]

In [ ]:
total[pd.isnull(total['cus_lat'])][['customer_zip_code_prefix','customer_city','customer_state','cus_lat','cus_lng']]

,customer_zip_code_prefix,customer_city,customer_state,cus_lat,cus_lng


In [ ]:
total[total['customer_zip_code_prefix']=='410'][['cus_lat','cus_lng']] 

,cus_lat,cus_lng
34246,-12.959735,-38.456443


In [ ]:
total[total['customer_zip_code_prefix']=='201'][['cus_lat','cus_lng']] 

,cus_lat,cus_lng
75616,-22.916367,-43.205141


### - geoloc seller와 prefix로 결합

In [ ]:
geoloc_seller = geoloc.groupby('zip_code_prefix').mean()
geoloc_seller.reset_index(inplace=True)
geoloc_seller.rename(columns={'zip_code_prefix':'seller_zip_code_prefix','lat': 'sel_lat','lng':'sel_lng'}, inplace=True)
geoloc_seller.head()

,seller_zip_code_prefix,sel_lat,sel_lng
0,010,-23.545414,-46.637157
1,011,-23.526827,-46.650764
2,012,-23.541328,-46.656093
3,013,-23.557785,-46.648632
4,014,-23.568757,-46.666038


In [ ]:
total = total.merge(geoloc_seller,on='seller_zip_code_prefix', how='left')

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
total[pd.isnull(total['sel_lat'])][['seller_zip_code_prefix','seller_city','seller_state','sel_lat','sel_lng']]

,seller_zip_code_prefix,seller_city,seller_state,sel_lat,sel_lng
424,NaN,NaN,NaN,NaN,NaN
31102,NaN,NaN,NaN,NaN,NaN
34009,NaN,NaN,NaN,NaN,NaN
37231,NaN,NaN,NaN,NaN,NaN
37665,NaN,NaN,NaN,NaN,NaN
39557,NaN,NaN,NaN,NaN,NaN
39662,NaN,NaN,NaN,NaN,NaN
39709,NaN,NaN,NaN,NaN,NaN
68320,NaN,NaN,NaN,NaN,NaN
75111,NaN,NaN,NaN,NaN,NaN


In [ ]:
total[['cus_lat','cus_lng','sel_lat','sel_lng']].head()

,cus_lat,cus_lng,sel_lat,sel_lng
0,-25.550950,-48.466168,-25.462300,-49.300827
1,-19.900549,-44.005775,-22.679820,-50.453874
2,-20.738866,-41.124768,-23.657011,-46.540121
3,-23.670007,-46.448906,-23.657011,-46.540121
4,-22.965741,-43.395909,-29.677045,-51.060403


In [ ]:
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93000 entries, 0 to 92999
Data columns (total 144 columns):
order_id                         93000 non-null object
order_status                     93000 non-null object
order_products_value             93000 non-null float64
order_freight_value              93000 non-null float64
order_items_qty                  93000 non-null int64
order_sellers_qty                93000 non-null int64
order_purchase_timestamp         93000 non-null datetime64[ns]
order_aproved_at                 92982 non-null datetime64[ns]
order_estimated_delivery_date    93000 non-null datetime64[ns]
order_delivered_customer_date    90705 non-null datetime64[ns]
product_id                       93000 non-null object
product_category_name_english    93000 non-null object
product_name_lenght              93000 non-null int64
product_description_lenght       93000 non-null int64
product_photos_qty               93000 non-null int64
product_weight_g                 929

## 파생변수 20. order_product_value

In [ ]:
total['order_products_value'].head()

0    349.90
1     60.00
2     79.99
3     58.99
4    639.00
Name: order_products_value, dtype: float64

In [ ]:
total['order_items_qty'].value_counts()

1     86503
2      4993
3       837
4       373
5       142
6       129
20        4
15        4
7         4
10        3
8         3
9         2
14        1
12        1
11        1
Name: order_items_qty, dtype: int64

In [ ]:
total['order_product_value'] = total['order_products_value']/total['order_items_qty']

## 파생변수 18. 'review_length' : 리뷰길이

In [ ]:
total['review_length'] = [len(i) if pd.notnull(i) else i for i in total['review_comment_message']]

In [ ]:
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93000 entries, 0 to 92999
Data columns (total 146 columns):
order_id                         93000 non-null object
order_status                     93000 non-null object
order_products_value             93000 non-null float64
order_freight_value              93000 non-null float64
order_items_qty                  93000 non-null int64
order_sellers_qty                93000 non-null int64
order_purchase_timestamp         93000 non-null datetime64[ns]
order_aproved_at                 92982 non-null datetime64[ns]
order_estimated_delivery_date    93000 non-null datetime64[ns]
order_delivered_customer_date    90705 non-null datetime64[ns]
product_id                       93000 non-null object
product_category_name_english    93000 non-null object
product_name_lenght              93000 non-null int64
product_description_lenght       93000 non-null int64
product_photos_qty               93000 non-null int64
product_weight_g                 929

# -----------------------------------------------------------------------------

In [ ]:
total[['review_comment_title', 'review_comment_message']].head()

,review_comment_title,review_comment_message
0,NaN,NaN
1,NaN,muito bom cabelo fica lisinho
2,Bom,gostei
3,NaN,NaN
4,NaN,o produto foi entregue no prazo e bem acondici...


## 파생변수 20. 리뷰 타이틀, 메시지 영문 

### - 리뷰 영어 번역 불러오기

In [ ]:
trans= pd.read_csv('./data_merged/final_dataset_translate.csv',skiprows=[0,1])

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3018: DtypeWarning: Columns (68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
review_message_english = trans[['order_id','review_comment_title','review_comment_message']]
review_message_english.rename(columns={'review_comment_message':'review_comment_message_english'},inplace=True)
review_message_english.rename(columns={'review_comment_title':'review_comment_title_english'},inplace=True)

C:\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [ ]:
review_message_english.head()

,order_id,review_comment_title_english,review_comment_message_english
0,b95df3cef5297e79ef709ba256518f6f,NaN,NaN
1,8a723730400b508cbf47fbef4a76ec8e,NaN,very good hair stays straight
2,cf71b9b6b7ed12d72f541b0420023f19,Good,liked it
3,93ef9253c47e7dfc633c01f728488cb1,NaN,NaN
4,35bc592a16d6706d14d29f07ed1d13c1,NaN,the product was delivered on time and well


In [ ]:
total = total.merge(review_message_english, on='order_id')

In [ ]:
total[['order_id','review_comment_title','review_comment_title_english','review_comment_message','review_comment_message_english']].head()

,order_id,review_comment_title,review_comment_title_english,review_comment_message,review_comment_message_english
0,b95df3cef5297e79ef709ba256518f6f,NaN,NaN,NaN,NaN
1,8a723730400b508cbf47fbef4a76ec8e,NaN,NaN,muito bom cabelo fica lisinho,very good hair stays straight
2,cf71b9b6b7ed12d72f541b0420023f19,Bom,Good,gostei,liked it
3,93ef9253c47e7dfc633c01f728488cb1,NaN,NaN,NaN,NaN
4,35bc592a16d6706d14d29f07ed1d13c1,NaN,NaN,o produto foi entregue no prazo e bem acondici...,the product was delivered on time and well


In [ ]:
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93000 entries, 0 to 92999
Data columns (total 148 columns):
order_id                          93000 non-null object
order_status                      93000 non-null object
order_products_value              93000 non-null float64
order_freight_value               93000 non-null float64
order_items_qty                   93000 non-null int64
order_sellers_qty                 93000 non-null int64
order_purchase_timestamp          93000 non-null datetime64[ns]
order_aproved_at                  92982 non-null datetime64[ns]
order_estimated_delivery_date     93000 non-null datetime64[ns]
order_delivered_customer_date     90705 non-null datetime64[ns]
product_id                        93000 non-null object
product_category_name_english     93000 non-null object
product_name_lenght               93000 non-null int64
product_description_lenght        93000 non-null int64
product_photos_qty                93000 non-null int64
product_weight_g     

In [ ]:
total.to_csv('./data_merged/final_dataset_var2.csv', index = False)

In [453]:
total_var = pd.read_csv('final_dataset_var2.csv')

/Users/YiYoung/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [109]:
total_var = total_var.drop(columns='Unnamed: 0')

In [455]:
total_var['credit_installments_1']

0         NaN
1         1.0
2         NaN
3         5.0
4        10.0
5         2.0
6         2.0
7         NaN
8         2.0
9         3.0
10       10.0
11        3.0
12        2.0
13        2.0
14        5.0
15        NaN
16        3.0
17        NaN
18        1.0
19        5.0
20        NaN
21        3.0
22        9.0
23        6.0
24       10.0
25        3.0
26        4.0
27        7.0
28        3.0
29        6.0
30        4.0
31        4.0
32        NaN
33        1.0
34        3.0
35        1.0
36        4.0
37        2.0
38        6.0
39        NaN
40        8.0
41        NaN
42        3.0
43        4.0
44        1.0
45        4.0
46        6.0
47        2.0
48        NaN
49        3.0
50        NaN
51        NaN
52        NaN
53       10.0
54        3.0
55        NaN
56        3.0
57        NaN
58        8.0
59        NaN
60        5.0
61        2.0
62        1.0
63        1.0
64        1.0
65        1.0
66        1.0
67        1.0
68        1.0
69        1.0
70        NaN
71    

In [ ]:
scaled_days = list(map(lambda abs_deli_days: abs_deli_days.total_seconds()/datetime.timedelta(days=1).total_seconds(), abs_deli_days))